# Project Set-up


In [162]:
try: 
    import pandas as pd
    import ta
    import numpy as np
    import requests
    import sys, os
    from dotenv import load_dotenv
    from pathlib import Path
    from datetime import datetime, timedelta
    
    print("Successfully Imported all the libraries")

except Exception as e: 
    print(f"Import Error: {e}")
    raise


Successfully Imported all the libraries


In [163]:
PROJECT_ROOT = Path('/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management')
DATA_DIR = PROJECT_ROOT/'data'

load_dotenv(PROJECT_ROOT/'.env')
print(".env file successfully loaded")


.env file successfully loaded


# Data Extraction: 
Extract S&P 500 healthcare data from wikipedia and financial modeling Prep Api

### Get Ticker data

In [164]:

alpha_vantage_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
etf_symbol = "DIA"

def get_qqq_holdings(api_key: str, symbol: str) -> pd.DataFrame:

    url = f'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200: 
        json_data = response.json()
        holdings_data = json_data["holdings"]
        df = pd.DataFrame(holdings_data)
        rows_to_drop = df[df['symbol'] == 'n/a'].index
        df = df.drop(rows_to_drop, axis = 0)

    else: 
        print("API request to Alpha Vantage failed.")
    df.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)
    return df


In [165]:
df = get_qqq_holdings(alpha_vantage_api_key, etf_symbol)
holdings_list = df['symbol'].to_list()
print(holdings_list)

['GS', 'MSFT', 'CAT', 'HD', 'SHW', 'UNH', 'V', 'AXP', 'JPM', 'MCD', 'IBM', 'AMGN', 'TRV', 'AAPL', 'CRM', 'AMZN', 'BA', 'HON', 'NVDA', 'JNJ', 'MMM', 'CVX', 'PG', 'DIS', 'WMT', 'MRK', 'NKE', 'CSCO', 'KO', 'VZ']


### Extract historical price data from financial modeling prep

In [166]:
base_url = "https://financialmodelingprep.com"
data_type = "historical-price-eod"
ticker = "AAPL"
FMP_API_KEY = os.getenv("FMP_API_KEY")

url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={FMP_API_KEY}"


In [167]:
def get_historical_price(url: str) -> pd.DataFrame: 
    try: 
        response = requests.get(url)
        print(response.status_code)
        data = response.json()
        df = pd.DataFrame(data, index = range(len(data)))
        print("Got historical price successfully")
    except Exception as e:
        print(f"There is an error: {e}")
        df = pd.DataFrame()
        

    

    return df

In [168]:
def get_historical_prices(tickers: list, api_key: str) -> pd.DataFrame:

    df = pd.DataFrame()
    for ticker in tickers:
        url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={api_key}"
        temp_df = get_historical_price(url)
        df = pd.concat([df,temp_df], axis = 0)

    # df.to_csv("/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management/data/raw/historical_price.csv", index = False)
    df.to_csv(DATA_DIR/"raw"/"historical_price.csv", index = False)
    return df

In [169]:
df = get_historical_prices(holdings_list, FMP_API_KEY)

200
Got historical price successfully
200
Got historical price successfully
402
There is an error: Expecting value: line 1 column 1 (char 0)
402
There is an error: Expecting value: line 1 column 1 (char 0)
402
There is an error: Expecting value: line 1 column 1 (char 0)
200
Got historical price successfully
200
Got historical price successfully
402
There is an error: Expecting value: line 1 column 1 (char 0)
200
Got historical price successfully
402
There is an error: Expecting value: line 1 column 1 (char 0)
402
There is an error: Expecting value: line 1 column 1 (char 0)
402
There is an error: Expecting value: line 1 column 1 (char 0)
402
There is an error: Expecting value: line 1 column 1 (char 0)
200
Got historical price successfully
402
There is an error: Expecting value: line 1 column 1 (char 0)
200
Got historical price successfully
200
Got historical price successfully
402
There is an error: Expecting value: line 1 column 1 (char 0)
200
Got historical price successfully
200
Got 

In [170]:
df.head(10)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap
0,GS,2025-10-06,795.51,801.665,780.0462,797.63,728025,2.12,0.266496,793.1100
1,GS,2025-10-03,783.00,794.920,777.8600,789.98,1470517,6.98,0.891440,786.4400
2,GS,2025-10-02,788.88,791.000,772.4400,779.38,1553849,-9.50,-1.200000,782.9250
3,GS,2025-10-01,793.62,796.280,783.2300,785.51,1789094,-8.11,-1.020000,789.6600
4,GS,2025-09-30,805.50,809.620,786.4500,796.35,2463934,-9.15,-1.140000,799.4800
5,GS,2025-09-29,807.80,808.840,797.9900,804.12,1059139,-3.68,-0.455560,804.6875
6,GS,2025-09-26,797.16,809.500,795.1300,802.51,1376242,5.35,0.671130,801.0750
7,GS,2025-09-25,786.00,798.180,782.0100,794.76,1413100,8.76,1.110000,790.2375
8,GS,2025-09-24,809.55,812.840,792.3400,792.43,1402500,-17.12,-2.110000,801.7900
9,GS,2025-09-23,807.10,825.250,802.0000,806.32,1759168,-0.78,-0.096642,810.1675


# Data Transformation

In [171]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

### Import the dataset

In [172]:
def load_data(filename): 
    df = pd.read_csv(filename)
    return df

In [173]:
# df = read_data(DATA_DIR)
df = load_data(DATA_DIR/"raw"/"historical_price.csv")
df.head(10)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap
0,GS,2025-10-06,795.51,801.665,780.0462,797.63,728025,2.12,0.266496,793.1100
1,GS,2025-10-03,783.00,794.920,777.8600,789.98,1470517,6.98,0.891440,786.4400
2,GS,2025-10-02,788.88,791.000,772.4400,779.38,1553849,-9.50,-1.200000,782.9250
3,GS,2025-10-01,793.62,796.280,783.2300,785.51,1789094,-8.11,-1.020000,789.6600
4,GS,2025-09-30,805.50,809.620,786.4500,796.35,2463934,-9.15,-1.140000,799.4800
5,GS,2025-09-29,807.80,808.840,797.9900,804.12,1059139,-3.68,-0.455560,804.6875
6,GS,2025-09-26,797.16,809.500,795.1300,802.51,1376242,5.35,0.671130,801.0750
7,GS,2025-09-25,786.00,798.180,782.0100,794.76,1413100,8.76,1.110000,790.2375
8,GS,2025-09-24,809.55,812.840,792.3400,792.43,1402500,-17.12,-2.110000,801.7900
9,GS,2025-09-23,807.10,825.250,802.0000,806.32,1759168,-0.78,-0.096642,810.1675


### Create Target Variable
1. Calculate weekly log returns

In [174]:
# Calculate weekly returns:
def calculate_weekly_returns(df: pd.DataFrame) -> pd.DataFrame:
    df['weekly_return'] = df.groupby('symbol')['close'].pct_change(periods = -5)
    df = df.dropna(subset = ["weekly_return"])
    
    df["weekly_log_return"] = np.log(1 + df["weekly_return"])
    
    return df

In [175]:
df = calculate_weekly_returns(df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_3004/387843821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weekly_log_return"] = np.log(1 + df["weekly_return"])


,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return
0,GS,2025-10-06,795.51,801.665,780.0462,797.63,728025,2.12,0.266496,793.110,-0.008071,-0.008104
1,GS,2025-10-03,783.00,794.920,777.8600,789.98,1470517,6.98,0.891440,786.440,-0.015614,-0.015737
2,GS,2025-10-02,788.88,791.000,772.4400,779.38,1553849,-9.50,-1.200000,782.925,-0.019352,-0.019541


2. 🎯 Create Target Variable

In [177]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    
    median_returns = df.groupby('year_week')['weekly_log_return'].median().reset_index()
    median_returns.columns = ['year_week', 'median_weekly_return']
    
    # Merge median returns back to original dataframe
    if 'median_weekly_return' not in df.columns:
        df = pd.merge(df, median_returns, on='year_week', how='left')
        
    df['target'] = (df['weekly_log_return'] > df['median_weekly_return']).astype(int)
    
    return df

In [178]:
df = create_target_variable(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
0,GS,2025-10-06,795.51,801.665,780.0462,797.63,728025,2.12,0.266496,793.110,-0.008071,-0.008104,2025-40,0.008162,0
1,GS,2025-10-03,783.00,794.920,777.8600,789.98,1470517,6.98,0.891440,786.440,-0.015614,-0.015737,2025-39,0.002083,0
2,GS,2025-10-02,788.88,791.000,772.4400,779.38,1553849,-9.50,-1.200000,782.925,-0.019352,-0.019541,2025-39,0.002083,0


In [179]:
# 🗂 Create the panel dataset indexed by date and tickers

def create_panel_dataset(df: pd.DataFrame) -> pd.DataFrame:
    return df.sort_values(by = "date")

In [180]:
# This step could be moved to the pipeline
df = create_panel_dataset(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
21249,VZ,2020-10-14,59.46,59.52,58.39,58.43,16596707,-1.03,-1.73,58.9500,-0.019795,-0.019994,2020-41,0.003059,0
1249,GS,2020-10-14,213.88,214.45,210.27,211.23,7557023,-2.65,-1.24,212.4575,0.037475,0.036790,2020-41,0.003059,1
16249,WMT,2020-10-14,48.80,48.84,47.92,47.98,19722346,-0.82,-1.68,48.3850,0.021721,0.021488,2020-41,0.003059,1


In [181]:
# 🗑 Remove columns that will not be used for the project
def remove_columns(cols: list, df: pd.DataFrame) -> pd.DataFrame:
    try: 
        df = df.drop(columns = cols, axis = 1)
    except Exception as e:
        print(e)
    finally: 
        return df

In [182]:
cols = ["open", "high", "low", "change", 
        "changePercent", "weekly_return", 
       "year_week", "median_weekly_return"]
df = remove_columns(cols, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target
21249,VZ,2020-10-14,58.43,16596707,58.9500,-0.019994,0
1249,GS,2020-10-14,211.23,7557023,212.4575,0.036790,1
16249,WMT,2020-10-14,47.98,19722346,48.3850,0.021488,1


2. Calculate moving averages

In [183]:
#📈 Moving Averages

def calc_moving_avg(window: int, df: pd.DataFrame) -> pd.DataFrame:
    col_name = f'MA_{str(window)}'
    df[col_name] = df.groupby('symbol')['close'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).mean()
    )
    
    return df


In [184]:
def calc_moving_avgs(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        df = calc_moving_avg(window, df)
    return df

In [185]:
windows = [200, 100, 50]
df = calc_moving_avgs(windows, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50
21249,VZ,2020-10-14,58.43,16596707,58.9500,-0.019994,0,58.43,58.43,58.43
1249,GS,2020-10-14,211.23,7557023,212.4575,0.036790,1,211.23,211.23,211.23
16249,WMT,2020-10-14,47.98,19722346,48.3850,0.021488,1,47.98,47.98,47.98


### Risk and Volatility

In [186]:
def calc_volatility(col_name: str, window: int, df: pd.DataFrame) -> pd.DataFrame: 
    df[col_name] = df.groupby('symbol')['weekly_log_return'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).std() * np.sqrt(window)
    )
    
    return df

In [187]:
def calc_volatilties(volatility_dict: dict, df: pd.DataFrame) -> pd.DataFrame: 
    for col_name, window in volatility_dict.items():
        df = calc_volatility(col_name, window, df)
    return df

In [189]:
volatility_dict = {
    'vol_1M': 4,    # 1 month ≈ 4 weeks
    'vol_6M': 26,   # 6 months ≈ 26 weeks
    'vol_12M': 52   # 12 months ≈ 52 weeks
}

df = calc_volatilties(volatility_dict, df)
#df = df.dropna(subset = ['vol_1M', 'vol_6M','vol_12M'])

df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M
21249,VZ,2020-10-14,58.43,16596707,58.9500,-0.019994,0,58.43,58.43,58.43,NaN,NaN,NaN
1249,GS,2020-10-14,211.23,7557023,212.4575,0.036790,1,211.23,211.23,211.23,NaN,NaN,NaN
16249,WMT,2020-10-14,47.98,19722346,48.3850,0.021488,1,47.98,47.98,47.98,NaN,NaN,NaN


### Short Term Reversal Factors

In [190]:
def calculate_rsi(window, group): 
    rsi = RSIIndicator(close = group['close'], window = window).rsi()
    return rsi

In [191]:
def calculate_rsis(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        col_name = f'RSI_{str(window)}'
        group = df.groupby('symbol', group_keys = False)
        df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
        
    return df

In [192]:
windows = [3, 9, 14]
df = calculate_rsis(windows, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_3004/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_3004/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k70

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14
21249,VZ,2020-10-14,58.43,16596707,58.9500,-0.019994,0,58.43,58.43,58.43,NaN,NaN,NaN,NaN,NaN,NaN
1249,GS,2020-10-14,211.23,7557023,212.4575,0.036790,1,211.23,211.23,211.23,NaN,NaN,NaN,NaN,NaN,NaN
16249,WMT,2020-10-14,47.98,19722346,48.3850,0.021488,1,47.98,47.98,47.98,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
def calculate_bb(group, window = 20, window_dev = 2):
    return BollingerBands(close = group['close'], window = window, window_dev = window_dev)

In [194]:
def calculate_bbs(bands: list, df:pd.DataFrame) -> pd.DataFrame:
    group = df.groupby('symbol', group_keys = False)
    for band in bands:
        if band == "hband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
        elif band == "lband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())
            
    return df

In [195]:
bands = ["hband", "lband"]
df = calculate_bbs(bands, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_3004/3629412943.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_3004/3629412943.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())


,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband
21249,VZ,2020-10-14,58.43,16596707,58.9500,-0.019994,0,58.43,58.43,58.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1249,GS,2020-10-14,211.23,7557023,212.4575,0.036790,1,211.23,211.23,211.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16249,WMT,2020-10-14,47.98,19722346,48.3850,0.021488,1,47.98,47.98,47.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Momentum Factor: 

In [196]:
def calculate_momentum(window_months: int, df: pd.DataFrame, trading_days :int = 21) -> pd.DataFrame:
    window_days = window_months * trading_days
    col_name = f"momentum_{str(window_months)}M"
    df[col_name] = df.groupby('symbol')['close'].pct_change(periods = window_days)
    
    return df

In [197]:
def calculate_momentums(windows: list, df:pd.DataFrame) -> pd.DataFrame:
    for window in windows:
        df = calculate_momentum(window, df)
    return df

In [198]:
windows_for_momentum = [12, 6, 1]

df = calculate_momentums(windows_for_momentum, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
21249,VZ,2020-10-14,58.43,16596707,58.9500,-0.019994,0,58.43,58.43,58.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1249,GS,2020-10-14,211.23,7557023,212.4575,0.036790,1,211.23,211.23,211.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16249,WMT,2020-10-14,47.98,19722346,48.3850,0.021488,1,47.98,47.98,47.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Datasets:

In [199]:
# 🗓Choose Each Wednesday and Thursday as the data transformation
def wed_thurs_selector(df, date_col='date', stock_col='symbol'):

    df['year_week'] = df[date_col].dt.strftime('%Y-%U')
    df['day_of_week'] = df[date_col].dt.dayofweek
    
    
    wed_thu = df[df['day_of_week'].isin([2, 3])].copy()
    
    
    filtered = wed_thu.groupby([stock_col, 'year_week']).first().reset_index()
    
    return filtered.drop(columns=['day_of_week', 'year_week'])
    

In [200]:
def filter_data(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy()
    df = wed_thurs_selector(df)
    return df.dropna(ignore_index = True)

In [201]:
df = filter_data(df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
0,AAPL,2021-10-13,140.91,78762721,140.6875,-0.007706,0,135.46275,141.8842,147.1556,...,0.115224,0.167709,24.298704,38.092347,40.057004,148.187287,138.687713,0.186236,0.047658,-0.048677
1,AAPL,2021-10-20,149.26,58418800,148.9575,0.057568,1,135.82010,142.9085,147.1822,...,0.139483,0.185654,93.598876,69.346276,61.062289,149.096687,138.388313,0.270190,0.131272,0.040647
2,AAPL,2021-10-27,148.85,56094929,149.1075,-0.002751,0,136.30825,144.1116,147.1786,...,0.126364,0.189954,56.427578,63.536072,58.586323,151.627006,137.969994,0.276587,0.115148,0.048904


In [202]:
def save_processed_data(df: pd.DataFrame, directory): 
    df.to_csv(directory/"processed"/"processed_historical_price.csv", index = False)

In [203]:
save_processed_data(df, DATA_DIR)

### Create Custom transformer
E.g. Remove/Fill in Nan values, log transformation, standardization

In [204]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

In [206]:
# Perform Log Transformation: 
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature] = np.log(X['close'] / X[feature])
        return X

### 💽Create training and testing variable

In [207]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold

import joblib

In [208]:
# Create x and y variable

def create_variables(df: pd.DataFrame) -> list: 
    return [df.drop(columns = ["target"]), df["target"]]

X, y = create_variables(df)


In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [211]:
# symbols = df["symbol"].unique().tolist()

# X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
# for symbol in symbols: 
#     symbol_df = df[df["symbol"] == symbol]
#     cut = int(len(symbol_df) * 0.8)
#     Xs, ys = create_variables(symbol_df)
#     Xs_train, Xs_test = Xs[:cut], Xs[cut:]
#     ys_train, ys_test = ys[:cut], ys[cut:]

#     X_train = pd.concat([X_train, Xs_train])
#     X_test = pd.concat([X_test, Xs_test])
#     y_train = pd.concat([y_train, ys_train])
#     y_test = pd.concat([y_test, ys_test])


def time_aware_split(df): 
    symbols = df["symbol"].unique().tolist()

    X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for symbol in symbols:
        symbol_df = df[df["symbol"] == symbol]
        cut = int(len(symbol_df) * 0.8)
        Xs, ys = create_variables(symbol_df)
        Xs_train, Xs_test = Xs[:cut], Xs[cut:]
        ys_train, ys_test = ys[:cut], ys[cut:]

        X_train = pd.concat([X_train, Xs_train])
        X_test = pd.concat([X_test, Xs_test])
        y_train = pd.concat([y_train, ys_train])
        y_test = pd.concat([y_test, ys_test])

    return X_train, X_test, y_train, y_test



In [212]:
X_train, X_test, y_train, y_test = time_aware_split(df)

In [213]:
# tscv = TimeSeriesSplit(n_splits=3)
# for i, (train_index, test_index) in enumerate(tscv.split(df)):
#     print(f"Fold {i + 1}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

### 🏭 Build a pipeline to transform features

In [214]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [216]:
#build pipeline
log_transform_features = ['MA_200', 'MA_100', 'MA_50', 'hband', 'lband']
# cols_to_drop = ['Ticker', 'date', 'weekly_log_return']

cols_to_drop = ['symbol', 'date', 'weekly_log_return']

# Example: Drop columns 'col1' and 'col2'
col_dropper = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', cols_to_drop)
    ],
    remainder='passthrough'  # keeps all other columns
)

logistic_regression_pipeline = Pipeline(steps = [ 
    ('log_transformers', LogTransformer(log_transform_features)),
    ('col_dropper', col_dropper), 
    ('scaler', StandardScaler()), 
    ('classifier', LogisticRegression())
])

In [217]:
logistic_regression_model = logistic_regression_pipeline.fit(X_train, y_train)

/Users/shengyaotang/opt/anaconda3/envs/bootcamp_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [218]:
filename = "logistic_regression_model.sav"
joblib.dump(logistic_regression_model, open(filename, 'wb'))

In [219]:
logistic_regression_model = joblib.load(open(filename, 'rb'))
y_pred = logistic_regression_model.predict(X_test)
print(y_pred)

[0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1.

In [220]:
print(f"Model accuracy: {logistic_regression_model.score(X_test, y_test)}")

Model accuracy: 0.7689075630252101


# Create Weekly Stock Portfolio

In [221]:
def get_last_week_data(filename):
    df = load_data(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    last_year_week = (datetime.now() - timedelta(weeks = 1)).strftime('%Y-%U')

    df_last_week = df[df['year_week'] == last_year_week]
    return df_last_week.drop(columns=['year_week'])

In [222]:
filename = DATA_DIR/'processed'/'processed_historical_price.csv'
df_last_week = get_last_week_data(filename).reset_index(drop = True)
X_last_week = df_last_week.drop(columns = ['target'])
y_last_week = df_last_week['target']


df_last_week

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
0,AAPL,2025-10-01,255.45,48713940,256.0525,0.012368,1,222.09110,217.9965,230.9796,...,0.165644,0.308144,67.857675,69.986470,68.822292,263.378832,224.568168,0.121428,0.144541,0.112006
1,AMZN,2025-10-01,220.63,43933834,219.1875,0.001905,0,214.65625,220.3845,226.9732,...,0.151001,0.251201,46.205832,39.409563,42.326179,240.410639,215.507361,0.173751,0.148098,-0.020902
2,BA,2025-10-01,215.20,7178000,214.6350,0.000465,0,195.11640,218.3300,225.6462,...,0.177758,0.231591,39.777869,41.765767,41.912128,231.904189,206.837811,0.376663,0.279657,-0.093437
3,CSCO,2025-10-01,68.75,15530716,68.2900,0.021019,1,63.50115,66.9503,68.1104,...,0.089228,0.193675,80.178113,60.633170,56.193948,68.838094,66.420906,0.296680,0.112460,0.014012
4,CVX,2025-10-01,154.58,6768400,154.4225,-0.029324,0,150.48075,150.1580,156.1584,...,0.117063,0.158850,17.252977,38.248692,43.583254,161.380428,153.103572,0.062479,-0.082666,-0.044800
5,DIS,2025-10-01,112.95,9736341,113.9700,-0.004241,0,109.99470,116.7606,116.5790,...,0.073434,0.156110,30.891485,38.842255,40.608273,119.172263,111.253737,0.176440,0.156327,-0.044982
6,GS,2025-10-01,785.51,1789094,789.6600,-0.008771,0,638.53110,699.6320,751.9984,...,0.132264,0.179004,18.913173,48.368470,55.058212,826.357101,742.705899,0.575716,0.432236,0.074790
7,JNJ,2025-10-01,186.05,13063800,185.6300,0.051619,1,159.77265,165.0788,175.3152,...,0.101029,0.155359,95.915823,77.352243,71.911888,184.229585,172.175415,0.152726,0.214029,0.044873
8,JPM,2025-10-01,310.71,9235211,311.6700,-0.008684,0,267.83850,287.4979,300.1822,...,0.096667,0.139656,17.418376,51.548075,57.064662,322.411049,294.564951,0.476057,0.275179,0.036737
9,KO,2025-10-01,66.78,14877538,66.6025,0.004954,1,68.55320,69.5302,68.3842,...,0.051905,0.105674,77.037085,50.107471,45.410860,68.302451,65.346549,-0.069787,-0.070822,-0.033015


In [223]:
y_pred = logistic_regression_model.predict(X_last_week)
y_prob = logistic_regression_model.predict_proba(X_last_week)

df_pred = pd.DataFrame(y_pred, columns = ['prediction'])
df_prob = pd.DataFrame(y_prob, columns = ['underperform', 'outperform'])

df_pred

,prediction
0,1.0
1,0.0
2,0.0
3,1.0
4,0.0
5,0.0
6,0.0
7,1.0
8,0.0
9,1.0


In [224]:
def create_weekly_stock_portfolio(*args: pd.DataFrame) -> pd.DataFrame:
    df_stock_portfolio = pd.concat(args, axis = 1)
    df_stock_portfolio = df_stock_portfolio.sort_values(by = ['outperform'], ascending = False)
    df_stock_portfolio = df_stock_portfolio[df_stock_portfolio['prediction'] == 1]
    df_stock_portfolio['weight'] = 1 / len(df_stock_portfolio)
    df_stock_portfolio.to_csv(DATA_DIR/'processed'/'stock_portfolio.csv', index = False)
    return df_stock_portfolio.reset_index(drop = True)

In [234]:
df_stock_portfolio = create_weekly_stock_portfolio(df_last_week['symbol'], df_prob, df_pred)
df_stock_portfolio

,symbol,underperform,outperform,prediction,weight
0,JNJ,0.045864,0.954136,1.0,0.1
1,NKE,0.062479,0.937521,1.0,0.1
2,NVDA,0.063328,0.936672,1.0,0.1
3,V,0.110352,0.889648,1.0,0.1
4,MSFT,0.133381,0.866619,1.0,0.1
5,CSCO,0.155454,0.844546,1.0,0.1
6,AAPL,0.190144,0.809856,1.0,0.1
7,UNH,0.196074,0.803926,1.0,0.1
8,KO,0.238954,0.761046,1.0,0.1
9,VZ,0.367374,0.632626,1.0,0.1


In [252]:
def update_stock_portfolio(df_stock_portfolio: pd.DataFrame, df_historical_price: pd.DataFrame) -> pd.DataFrame:
    df_historical_price = df_historical_price.copy()
    
    df_historical_price['date'] = pd.to_datetime(df_historical_price['date'])
    df_historical_price['week_of_year'] = df_historical_price['date'].dt.strftime('%Y-%U')
    df_weekly_price = df_historical_price[df_historical_price['week_of_year'] == datetime.now().strftime('%Y-%U') ]

    if 'close' not in df_stock_portfolio.columns: 
        df_weekly_portfolio = pd.merge(df_stock_portfolio, df_weekly_price, on = 'symbol', how = 'left')
    df_weekly_portfolio.to_csv(DATA_DIR/'processed'/'weekly_portfolio.csv')

    return df_weekly_portfolio.dropna()

In [253]:
df_historical_price = load_data(DATA_DIR/'raw'/'historical_price.csv')
df_weekly_portfolio = update_stock_portfolio(df_stock_portfolio, df_historical_price)
df_weekly_portfolio

,symbol,underperform,outperform,prediction,weight,date,open,high,low,close,volume,change,changePercent,vwap,week_of_year
0,JNJ,0.045864,0.954136,1.0,0.1,2025-10-06,188.000,189.914,187.5924,188.1250,2277281,0.1250,0.066489,188.54,2025-40
1,NKE,0.062479,0.937521,1.0,0.1,2025-10-06,72.020,72.390,71.0200,71.4450,7945811,-0.5750,-0.798389,71.62,2025-40
2,NVDA,0.063328,0.936672,1.0,0.1,2025-10-06,185.560,187.230,183.3300,186.0450,105610419,0.4850,0.261371,185.54,2025-40
3,V,0.110352,0.889648,1.0,0.1,2025-10-06,350.010,351.120,344.3500,348.6100,2433076,-1.4000,-0.399989,348.03,2025-40
4,MSFT,0.133381,0.866619,1.0,0.1,2025-10-06,518.610,525.390,518.2000,523.8140,7791033,5.2000,1.003450,522.47,2025-40
5,CSCO,0.155454,0.844546,1.0,0.1,2025-10-06,68.500,68.690,67.8100,68.2750,5180575,-0.2250,-0.328467,68.26,2025-40
6,AAPL,0.190144,0.809856,1.0,0.1,2025-10-06,257.945,259.070,255.0500,257.1805,20087195,-0.7645,-0.296381,257.10,2025-40
7,UNH,0.196074,0.803926,1.0,0.1,2025-10-06,361.025,362.645,357.1100,359.6100,4323674,-1.4100,-0.391940,359.79,2025-40
8,KO,0.238954,0.761046,1.0,0.1,2025-10-06,66.490,66.500,65.8850,65.9650,6578316,-0.5250,-0.789592,66.12,2025-40
9,VZ,0.367374,0.632626,1.0,0.1,2025-10-06,43.250,43.250,41.3500,41.4150,44454288,-1.8400,-4.242770,42.01,2025-40
